In [40]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [41]:
df_flange = pd.read_csv('data/flange.csv')

In [42]:
df_flange_train = df_flange.query("T == 'training'").drop(columns=['T','No','30','40','50','60','70','80','90'])
df_flange_test = df_flange.query("T == 'test'").drop(columns=['T','No','30','40','50','60','70','80','90'])

In [44]:
df_flange_test

,L,B,D,T.1,DWT,SC,OP,GP,WP,CREW,WD,TLWT,Total_flange
30,285.0,60.0,32.0,24.0,340660,2.0,0.0,400.0,1.0,100,1010,23000,1085
31,319.0,56.0,30.0,20.0,255271,2.0,0.0,71.0,0.0,110,1260,14500,1019
32,305.0,61.0,32.0,24.0,350000,2.0,0.0,250.0,0.0,240,1200,37478,1371


In [55]:
df_flange_30 = df_flange.loc[0:29,'30']
df_flange_40 = df_flange.loc[0:29,'40']
df_flange_50 = df_flange.loc[0:29,'50']
df_flange_60 = df_flange.loc[0:29,'60']
df_flange_70 = df_flange.loc[0:29,'70']
df_flange_80 = df_flange.loc[0:29,'80']
df_flange_90 = df_flange.loc[0:29,'90']

In [56]:
X = df_flange_train.values
Y = df_flange_30

In [57]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [58]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)

In [59]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold, n_jobs=1)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -1525902.35 (3560709.80) MSE


In [60]:
# evaluate model with standardized dataset
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold, n_jobs=1)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: -53.19 (43.50) MSE


In [61]:
# define the model
def larger_model():
	# create model
	model = Sequential()
	model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
	model.add(Dense(6, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [62]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold, n_jobs=1)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: -36.51 (31.64) MSE


In [63]:
# define wider model
def wider_model():
	# create model
	model = Sequential()
	model.add(Dense(20, input_dim=13, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [64]:
np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold, n_jobs=1)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider: -30.24 (25.95) MSE
